In [1]:
using InteractiveUtils
versioninfo()


Julia Version 1.10.9
Commit 5595d20a287 (2025-03-10 12:51 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake-avx512)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = auto


In [2]:
using Pkg
Pkg.add([
    "CSV", "DataFrames", "MLJ", "MLJFlux", "Flux", "Plots", "StatsBase",
    "CategoricalArrays", "MLJBase", "Glob", "DecisionTree", "MLJDecisionTreeInterface",
    "NearestNeighborModels", "MLJLinearModels", "LIBSVM", "EvoTrees", "LightGBM",
    "MLJLIBSVMInterface","MLJModels",
    "MLJModelInterface"
])


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed ScikitLearnBase ────────── v0.5.0
   Installed ShowCases ──────────────── v0.1.0
   Installed StatisticalTraits ──────── v3.4.0
   Installed NetworkLayout ──────────── v0.4.10
   Installed Accessors ──────────────── v0.1.42
   Installed LightGBM_jll ───────────── v3.3.5+1
   Installed ContextVariablesX ──────── v0.1.3
   Installed CategoricalDistributions ─ v0.1.15
   Installed InitialValues ──────────── v0.3.1
   Installed NearestNeighbors ───────── v0.4.21
   Installed ZygoteRules ────────────── v0.2.7
   Installed MLUtils ────────────────── v0.4.8
   Installed LearnAPI ───────────────── v1.0.1
   Installed EarlyStopping ──────────── v0.3.0
   Installed PrettyPrint ────────────── v0.2.0
   Installed NearestNeighborModels ──── v0.2.3
   Installed MLJModels ──────────────── v0.17.9
   Installed IterationControl ───────── v0.5.4
   Installed RealDot ────────────────── v0.1.0
   Inst

In [16]:
using CSV, DataFrames, MLJ, Flux, Plots, StatsBase, CategoricalArrays, MLJFlux

In [17]:
import Glob
path = "./cleaneddata"
files = Glob.glob("S*_cleaned_domain_features.csv", path)

df_all = DataFrame[]
for file in files
    push!(df_all, CSV.read(file, DataFrame))
end

data = reduce(vcat, df_all)
first(data, 5)


Row,sid,rMSSD,SDNN,pNN50,LF_frequency_peak,LF_frequency_power,HF_frequency_peak,HF_frequency_power,total_power,IBI_mean,IBI_median,IBI_max,IBI_min,IBI_std,HR_mean,HR_median,HR_max,HR_min,HR_std,HRV_ApEn,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC,circadian_cosine_start,circadian_cosine_decay,circadian_cosine_linear,Sleep_Stage
,String7,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String1
1,S002,291.254,198.942,64.6259,0.0916667,204.331,0.241667,483.371,5739.3,0.854366,0.84375,1.15625,0.671875,0.131768,0.214561,0.191428,0.390688,0.06712,0.105759,0.792569,1.20705,4.81094,1.04305,0.799375,1.3601,1.39029,1.32066,2.00169,3.33634,1.17546,0.0119814,0.98259,0.00381388,P
2,S002,386.275,286.862,82.9932,0.0833333,121.643,0.158333,401.012,4206.22,0.739913,0.734375,0.859375,0.734375,0.0146324,0.54212,0.210623,1.72609,0.12379,0.518313,0.731853,1.82735,5.66427,1.46808,1.26058,1.17599,1.53516,1.92788,1.99834,4.17921,1.0775,0.023961,0.965483,0.00762777,P
3,S002,310.197,218.362,67.3203,0.0833333,163.949,0.241667,469.285,4542.46,0.765466,0.734375,0.96875,0.734375,0.0743457,0.911839,0.785093,1.78276,0.288316,0.550858,0.666223,0.996238,5.03294,1.08203,1.29357,1.16307,1.22496,1.38469,1.98497,3.4982,1.09098,0.0359373,0.948673,0.0114416,P
4,S002,318.613,249.197,69.9301,0.0916667,280.756,0.241667,193.683,4581.74,0.768691,0.78125,0.90625,0.59375,0.0754652,0.437562,0.425421,0.546987,0.354126,0.040068,0.749222,0.966588,5.10161,1.23342,1.14552,1.2085,1.40636,1.40856,1.96053,3.46928,1.10152,0.0479083,0.932157,0.0152555,P
5,S002,304.647,206.281,71.8121,0.1,338.04,0.191667,620.805,5660.8,0.854274,0.90625,1.01562,0.640625,0.0854323,0.0896814,0.115564,0.423593,-0.246393,0.243694,0.710629,1.29152,5.22051,1.25967,1.20749,1.39171,1.58391,1.51793,1.99466,4.39993,1.16282,0.0598725,0.915928,0.0190694,P


In [18]:
if "sid" in names(data)
    select!(data, Not("sid"))
end

foreach(col -> replace!(col, missing => 0.0), eachcol(data))

valid_stages = ["W", "N", "R"]
data = filter(row -> row.Sleep_Stage in valid_stages, data)

label_mapping = Dict("W"=> "Wake", "N"=> "NREM", "R"=> "REM")
data.Sleep_Stage = CategoricalArray([label_mapping[String(x)] for x in data.Sleep_Stage])

y = data.Sleep_Stage
X = select(data, Not(:Sleep_Stage))
non_constant_cols = names(X)[std.(eachcol(Matrix(X))) .> 0.0]
X = select(X, non_constant_cols)

Row,rMSSD,SDNN,pNN50,LF_frequency_peak,LF_frequency_power,HF_frequency_peak,HF_frequency_power,total_power,IBI_mean,IBI_median,IBI_max,IBI_min,IBI_std,HR_mean,HR_median,HR_max,HR_min,HR_std,HRV_ApEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC,circadian_cosine_start,circadian_cosine_decay,circadian_cosine_linear
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,485.018,433.173,87.9032,0.0916667,396.591,0.241667,673.864,6058.27,0.885388,0.953125,0.953125,0.8125,0.0701784,1.02479,1.06524,1.42354,0.408968,0.226097,0.931941,5.70279,1.35855,1.18724,1.4379,1.82889,1.6382,1.9931,1.9935,1.12164,0.793536,0.260901,0.291762
2,422.248,314.127,85.6061,0.1,130.599,0.233333,639.783,5492.36,0.842592,0.8125,0.96875,0.765625,0.0720376,1.19044,1.12466,1.57436,0.952818,0.171888,0.778819,5.74297,1.65644,1.38917,1.38225,1.79073,1.87722,1.99126,3.48336,1.17407,0.795355,0.259758,0.292715
3,395.127,297.914,86.3636,0.0916667,177.055,0.225,517.887,4873.61,0.795717,0.8125,0.96875,0.765625,0.0376666,1.01138,1.06799,1.57436,-0.115687,0.386523,0.580465,5.70448,1.70338,1.42043,1.55365,1.95231,1.77674,2.02138,6.22896,1.1207,0.797167,0.25862,0.293669
4,406.78,320.998,86.0465,0.1,174.384,0.15,288.63,4695.33,0.781325,0.765625,0.96875,0.765625,0.0300324,0.694124,1.0072,1.57436,-0.351507,0.666761,0.745459,5.77364,1.62205,1.60142,1.32257,1.72526,1.82947,2.0025,4.41651,1.08701,0.798972,0.257487,0.294622
5,512.473,380.493,88.8889,0.1,275.937,0.225,373.757,4553.3,0.769607,0.765625,0.96875,0.765625,0.0241511,0.487202,0.403484,1.57436,-0.351507,0.626374,0.679913,5.90539,1.57948,1.61003,1.23545,1.7037,1.7403,2.00819,3.05781,1.05213,0.80077,0.256359,0.295576
6,518.03,365.844,90.5512,0.0666667,196.242,0.241667,381.979,4501.88,0.765625,0.765625,0.765625,0.765625,0.0,0.351323,0.403484,0.994863,-0.351507,0.444787,0.626747,5.87608,1.62676,1.30132,1.30742,1.59823,1.8381,2.02379,2.87786,1.10058,0.80256,0.255236,0.296529
7,479.798,342.976,91.6667,0.1,273.464,0.225,397.793,4501.88,0.765625,0.765625,0.765625,0.765625,0.0,0.427279,0.69506,0.882437,-0.351507,0.456711,0.586445,5.87467,1.67177,1.45526,1.35657,1.60832,2.20228,2.02752,5.91963,1.17407,0.804344,0.254117,0.297483
8,458.806,332.677,90.5109,0.0916667,196.37,0.183333,564.007,4501.88,0.765625,0.765625,0.765625,0.765625,0.0,0.655353,0.750359,0.882437,-0.0791252,0.247858,0.7382,5.81818,1.65621,1.54826,1.1698,1.65546,1.95841,2.01497,3.94875,1.29526,0.80612,0.253004,0.298436
9,434.523,318.434,90.5797,0.1,232.228,0.233333,485.525,4501.88,0.765625,0.765625,0.765625,0.765625,0.0,0.660289,0.743504,0.882437,0.231646,0.203656,0.82315,5.71539,1.63474,1.57396,1.31778,1.78394,2.06664,2.01801,3.99746,1.18475,0.807889,0.251896,0.29939


In [19]:
using MLJ, MLJFlux, Flux, StatsBase, DataFrames, CategoricalArrays
using DecisionTree, NearestNeighborModels, MLJLinearModels, LIBSVM, EvoTrees

@load DecisionTreeClassifier pkg=DecisionTree
@load KNNClassifier pkg=NearestNeighborModels
@load LogisticClassifier pkg=MLJLinearModels
@load SVC pkg=LIBSVM
@load EvoTreeClassifier pkg=EvoTrees

input_dim = size(X, 2)
output_dim = length(levels(y))

deep_net = Chain(
    Dense(input_dim, 128, relu),
    Dense(128, 64, relu),
    Dense(64, output_dim),
    Flux.softmax
)

deep_builder = MLJFlux.GenericBuilder(deep_net)
cv = CV(nfolds=3, shuffle=true)

import MLJDecisionTreeInterface ✔
import NearestNeighborModels ✔
import MLJLinearModels ✔
import MLJLIBSVMInterface ✔
import EvoTrees ✔


[ Info: For silent loading, specify `verbosity=0`. 
[ Info: For silent loading, specify `verbosity=0`. 
[ Info: For silent loading, specify `verbosity=0`. 
[ Info: For silent loading, specify `verbosity=0`. 
[ Info: For silent loading, specify `verbosity=0`. 


CV(
  nfolds = 3, 
  shuffle = true, 
  rng = Random._GLOBAL_RNG())

In [9]:
model = Standardizer() |> DecisionTreeClassifier(max_depth=3)

println("\n Evaluating: ", model)
try
    mach = machine(model, X, y)
    res = evaluate!(mach, resampling=cv,
        measures=[accuracy, macro_f1score, micro_f1score, cross_entropy],
        verbosity=0)
    acc = round(mean(res.measurement[1]), digits=4)
    println("→ Accuracy: ", acc)

    MLJ.fit!(mach)
    model_name = string(typeof(model).name.name)
    filename = model_name * ".jls"
    MLJ.save(filename, mach)
    println("Saved model to $filename")

    ŷ = predict_mode(mach, X)
    cm = confusion_matrix(y, ŷ)
    println("Confusion Matrix:\n", cm)
catch err
    println("Error with model $(model): ", err)
end


 Evaluating: UnsupervisedPipeline(standardizer = Standardizer(features = Symbol[], …), …)
Error with model UnsupervisedPipeline(standardizer = Standardizer(features = Symbol[], …), …): ArgumentError("\n The `prediction_type` of your model needs to be one of: `:deterministic`,\n `:probabilistic`, or `:interval`. Does your model implement one of these operations:\n `predict`, `predict_mean`, `predict_mode`, `predict_median`, or `predict_joint`? If so, you can try explicitly specifying `operation=...`\n or `operations=...` (and consider posting an issue to have the model review it's\n definition of `MLJModelInterface.prediction_type`). Otherwise, performance\n evaluation is not supported.\n\n")


In [ ]:
model = Standardizer() |> KNNClassifier(K=5)

println("\n Evaluating: ", model)
try
    mach = machine(model, X, y)
    res = evaluate!(mach, resampling=cv,
        measures=[accuracy, macro_f1score, micro_f1score, cross_entropy],
        verbosity=0)
    acc = round(mean(res.measurement[1]), digits=4)
    println("→ Accuracy: ", acc)

    MLJ.fit!(mach)
    model_name = string(typeof(model).name.name)
    filename = model_name * ".jls"
    MLJ.save(filename, mach)
    println("Saved model to $filename")

    ŷ = predict_mode(mach, X)
    cm = confusion_matrix(y, ŷ)
    println("Confusion Matrix:\n", cm)
catch err
    println("Error with model $(model): ", err)
end


 Evaluating: ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …)


┌ Warning: Extremely small standard deviation encountered in standardization.
└ @ MLJModels ~/.julia/packages/MLJModels/nxeCf/src/builtins/Transformers.jl:425
┌ Warning: Extremely small standard deviation encountered in standardization.
└ @ MLJModels ~/.julia/packages/MLJModels/nxeCf/src/builtins/Transformers.jl:425
┌ Warning: Extremely small standard deviation encountered in standardization.
└ @ MLJModels ~/.julia/packages/MLJModels/nxeCf/src/builtins/Transformers.jl:425


→ Accuracy: 0.6102


[ Info: Training machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
┌ Warning: Extremely small standard deviation encountered in standardization.
└ @ MLJModels ~/.julia/packages/MLJModels/nxeCf/src/builtins/Transformers.jl:425
[ Info: Training machine(:knn_classifier, …).


In [20]:
model = Standardizer() |> LogisticClassifier()

println("\n Evaluating: ", model)
try
    mach = machine(model, X, y)
    res = evaluate!(mach, resampling=cv,
        measures=[accuracy, macro_f1score, micro_f1score, cross_entropy],
        verbosity=0)
    acc = round(mean(res.measurement[1]), digits=4)
    println("Accuracy: ", acc)

    MLJ.fit!(mach)
    model_name = string(typeof(model).name.name)
    filename = model_name * ".jls"
    MLJ.save(filename, mach)
    println("Saved model to $filename")

    ŷ = predict_mode(mach, X)
    cm = confusion_matrix(y, ŷ)
    println("Confusion Matrix:\n", cm)
catch err
    println("Error with model $(model): ", err)
end


 Evaluating: ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …)
→ Accuracy: 0.7109


[ Info: Training machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:logistic_classifier, …).
┌ Info: Solver: LBFGS{Optim.Options{Float64, Nothing}, @NamedTuple{}}
│   optim_options: Optim.Options{Float64, Nothing}
└   lbfgs_options: @NamedTuple{} NamedTuple()


Saved model to ProbabilisticPipeline.jls
Confusion Matrix:
ConfusionMatrix{3}([30397 470 2481; 5399 805 672; 6551 201 7676])


In [21]:
model = Standardizer() |> SVC()

println("\n Evaluating: ", model)
try
    mach = machine(model, X, y)
    res = evaluate!(mach, resampling=cv,
        measures=[accuracy, macro_f1score, micro_f1score, cross_entropy],
        verbosity=0)
    acc = round(mean(res.measurement[1]), digits=4)
    println("Accuracy: ", acc)

    MLJ.fit!(mach)
    model_name = string(typeof(model).name.name)
    filename = model_name * ".jls"
    MLJ.save(filename, mach)
    println("Saved model to $filename")

    ŷ = predict_mode(mach, X)
    cm = confusion_matrix(y, ŷ)
    println("Confusion Matrix:\n", cm)
catch err
    println("Error with model $(model): ", err)
end


 Evaluating: UnsupervisedPipeline(standardizer = Standardizer(features = Symbol[], …), …)
Error with model UnsupervisedPipeline(standardizer = Standardizer(features = Symbol[], …), …): ArgumentError("\n The `prediction_type` of your model needs to be one of: `:deterministic`,\n `:probabilistic`, or `:interval`. Does your model implement one of these operations:\n `predict`, `predict_mean`, `predict_mode`, `predict_median`, or `predict_joint`? If so, you can try explicitly specifying `operation=...`\n or `operations=...` (and consider posting an issue to have the model review it's\n definition of `MLJModelInterface.prediction_type`). Otherwise, performance\n evaluation is not supported.\n\n")


In [14]:
model = Standardizer() |> EvoTreeClassifier(nrounds=100)

println("\n Evaluating: ", model)
try
    mach = machine(model, X, y)
    res = evaluate!(mach, resampling=cv,
        measures=[accuracy, macro_f1score, micro_f1score, cross_entropy],
        verbosity=0)
    acc = round(mean(res.measurement[1]), digits=4)
    println("Accuracy: ", acc)

    MLJ.fit!(mach)
    model_name = string(typeof(model).name.name)
    filename = model_name * ".jls"
    MLJ.save(filename, mach)
    println("Saved model to $filename")

    ŷ = predict_mode(mach, X)
    cm = confusion_matrix(y, ŷ)
    println("Confusion Matrix:\n", cm)
catch err
    println("Error with model $(model): ", err)
end


 Evaluating: ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …)


┌ Warning: Extremely small standard deviation encountered in standardization.
└ @ MLJModels ~/.julia/packages/MLJModels/nxeCf/src/builtins/Transformers.jl:425
┌ Error: Problem fitting the machine machine(:evo_tree_classifier, …). 
└ @ MLJBase ~/.julia/packages/MLJBase/F1Eh6/src/machines.jl:694
[ Info: Running type checks... 
[ Info: Type checks okay. 
┌ Error: Problem fitting machine(:evo_tree_classifier, …)
└ @ MLJBase ~/.julia/packages/MLJBase/F1Eh6/src/machines.jl:766
┌ Error: Problem fitting the machine machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …). 
└ @ MLJBase ~/.julia/packages/MLJBase/F1Eh6/src/machines.jl:694
[ Info: Running type checks... 


Error with model ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …): CompositeException(Any

[ Info: Type checks okay. 


[TaskFailedException(Task (failed) @0x00007df54b994fb0)])


In [15]:
model = Standardizer() |> NeuralNetworkClassifier(
    builder=deep_builder,
    loss=Flux.Losses.crossentropy,
    optimiser=Flux.ADAM(),
    epochs=20
)
println("\n Evaluating: ", model)
try
    mach = machine(model, X, y)
    res = evaluate!(mach, resampling=cv,
        measures=[accuracy, macro_f1score, micro_f1score, cross_entropy],
        verbosity=0)
    acc = round(mean(res.measurement[1]), digits=4)
    println("Accuracy: ", acc)

    MLJ.fit!(mach)
    model_name = string(typeof(model).name.name)
    filename = model_name * ".jls"
    MLJ.save(filename, mach)
    println("Saved model to $filename")

    ŷ = predict_mode(mach, X)
    cm = confusion_matrix(y, ŷ)
    println("Confusion Matrix:\n", cm)
catch err
    println("Error with model $(model): ", err)
end


 Evaluating: ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …)


┌ Warning: Extremely small standard deviation encountered in standardization.
└ @ MLJModels ~/.julia/packages/MLJModels/nxeCf/src/builtins/Transformers.jl:425


Error with model ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …): CompositeException(Any[TaskFailedException(Task (failed) @0x00007df585cd0fb0)])


┌ Error: Builder does not appear to build an architecture compatible with supplied data. 
└ @ MLJFlux ~/.julia/packages/MLJFlux/5eWpt/src/fit_utils.jl:14
┌ Error: Problem fitting the machine machine(:neural_network_classifier, …). 
└ @ MLJBase ~/.julia/packages/MLJBase/F1Eh6/src/machines.jl:694
[ Info: Running type checks... 
[ Info: Type checks okay. 
┌ Error: Problem fitting machine(:neural_network_classifier, …)
└ @ MLJBase ~/.julia/packages/MLJBase/F1Eh6/src/machines.jl:766
┌ Error: Problem fitting the machine machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …). 
└ @ MLJBase ~/.julia/packages/MLJBase/F1Eh6/src/machines.jl:694
[ Info: Running type checks... 
[ Info: Type checks okay. 
